ref: Kyle slack:
@Mark Kogan I had a conversation with @Adam Kupczyk about column families for omap, and as it turns out we actually use 3 by default. This is controlled by the bluestore_rocksdb_cfs parameter, by setting values like p(3,0-12), where the 3 indicates the number of column families and 0-13 indicates which character positions of the pg to use to hash a particular onode to a particular column family. I started on this path by looking into how other distributed databases use column families for a given table or collection of columns. My thinking is that a bucket index, or even an index shard, is more or less a table or collection of columns. CockroachDB uses a distinct column family for a given table, but also has the ability to break up collections of columns in a single table across multiple column families as an optimization [1]. The main implication of increasing the number of column families is that it will increase the amount of memory used by the OSDs because there is a set of memtables for each column family. The other thing about this parameter is that it is only used during OSD mkfs, on restart OSDs determine how many column families to use by looking at how many currently exist on disk. It would be interesting to experiment with both lower values (ie. p(1,0-12)), but also higher values (perhaps p(6,0-12), p(12,0-12 or similar).
https://www.cockroachlabs.com/docs/stable/column-families -- Column Families
- name: bluestore_rocksdb_cf
  desc: Enable use of rocksdb column families for bluestore metadata
  fmt_desc: Enables sharding of BlueStore's RocksDB.
    When ``true``, ``bluestore_rocksdb_cfs`` is used.
    Only applied when OSD is doing ``--mkfs``.
  default: true
- name: bluestore_rocksdb_cfs
  desc: Definition of column families and their sharding
  long_desc: 'Space separated list of elements: column_def [ ''='' rocksdb_options
    ]. column_def := column_name [ ''('' shard_count [ '','' hash_begin ''-'' [ hash_end
    ] ] '')'' ]. Example: ''I=write_buffer_size=1048576 O(6) m(7,10-)''. Interval
    [hash_begin..hash_end) defines characters to use for hash calculation. Recommended
    hash ranges: O(0-13) P(0-8) m(0-16). Sharding of S,T,C,M,B prefixes is inadvised'
  fmt_desc: Definition of BlueStore's RocksDB sharding.
    The optimal value depends on multiple factors, and modification is inadvisable.
    This setting is used only when OSD is doing ``--mkfs``.
    Next runs of OSD retrieve sharding from disk.
  default: m(3) p(3,0-12) O(3,0-13)=block_cache={type=binned_lru} L=min_write_buffer_number_to_merge=32 P=min_write_buffer_number_to_merge=32

In [ ]:
%pip install pandas

import pandas as pd

# Load the file into a dataframe

df_cf_ref_p3 = pd.read_csv('./hsbench_1731352724--ref.log')
print("print(df_cf_ref_p3)")
print(df_cf_ref_p3)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ParserError: Error tokenizing data. C error: Expected 1 fields in line 19, saw 12


In [ ]:
with open('./hsbench_1731352724--ref.log', 'r') as infile:
    lines = infile.readlines()

filtered_lines = [line for line in lines if 'TOTAL' in line and 'BINIT' not in line and 'PUT' in line]

with open('filtered_results.csv', 'w') as outfile:
    outfile.writelines(filtered_lines)